<a href="https://colab.research.google.com/github/johnbont/Clearliving2/blob/main/intent_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Always-Listening AI Device Intent Classifier
## ModernBERT Training on Google Colab

This notebook trains a ModernBERT-based intent classifier to determine when an always-listening AI device should respond to user speech. The model predicts confidence scores (0.0-1.0) for how likely the AI should help with a given utterance.

**Dataset:** Synthetic conversations with realistic living room/office scenarios including:
- Direct commands to the AI
- Conversations between people where AI could help
- Social chat where AI should stay silent
- Uses name placeholders: [name], [partner], [friend], etc.

## **Decision Thresholds (configurable):**
- >=0.85: High confidence - Voice + Visual response
- >=0.7: High - Should definitely help
- >=0.55: Medium-high - Potentially helpful
- >=0.35: Medium - Ambiguous cases
- >=0.15: Low - Probably shouldn't help
- <0.15: Very low - Ignore/stay silent

**Model Output:** Continuous confidence score (0.0-1.0) for flexible threshold tuning

In [ ]:
## 1. Setup and Installation


In [1]:
# Install required packages
!pip install torch transformers datasets scikit-learn pandas numpy tqdm matplotlib seaborn


In [2]:
import torch
import torch.nn as nn
from transformers import (
    AutoTokenizer, AutoModel, AutoConfig,
    TrainingArguments, Trainer, EvalPrediction
)
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, mean_squared_error
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from typing import Dict, List, Tuple
from tqdm import tqdm
import json
import os

# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)

# Check GPU availability
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")


Using device: cuda
GPU: NVIDIA A100-SXM4-40GB


In [3]:
from google.colab import files

# Upload the dataset
print("Please upload your synthetic_intent_v1_.csv file:")
uploaded = files.upload()

# Load the data
df = pd.read_csv('synthetic_intent_v1_oai.csv')
print(f"Dataset loaded: {df.shape}")
print(f"Columns: {list(df.columns)}")
print(f"\nLabel distribution:")
print(df['label'].describe())

# Show confidence score distribution
print(f"\nConfidence Score Ranges:")
very_high = len(df[df['label'] >= 0.85])
high = len(df[(df['label'] >= 0.7) & (df['label'] < 0.85)])
medium_high = len(df[(df['label'] >= 0.55) & (df['label'] < 0.7)])
medium = len(df[(df['label'] >= 0.35) & (df['label'] < 0.55)])
low = len(df[(df['label'] >= 0.15) & (df['label'] < 0.35)])
very_low = len(df[df['label'] < 0.15])

print(f"  Very High (0.85-1.0): {very_high:,} ({very_high/len(df)*100:.1f}%)")
print(f"  High (0.7-0.85): {high:,} ({high/len(df)*100:.1f}%)")
print(f"  Medium-High (0.55-0.7): {medium_high:,} ({medium_high/len(df)*100:.1f}%)")
print(f"  Medium (0.35-0.55): {medium:,} ({medium/len(df)*100:.1f}%)")
print(f"  Low (0.15-0.35): {low:,} ({low/len(df)*100:.1f}%)")
print(f"  Very Low (0.0-0.15): {very_low:,} ({very_low/len(df)*100:.1f}%)")

# Show some examples
print(f"\nSample conversations:")
sample_df = df.sample(n=10).sort_values('label', ascending=False)
for _, row in sample_df.iterrows():
    print(f"  Score {row['label']:.2f}: \"{row['text']}\"")


Please upload your synthetic_intent_v1_.csv file:


Saving synthetic_intent_v1_oai.csv to synthetic_intent_v1_oai.csv
Dataset loaded: (10726, 2)
Columns: ['text', 'label']

Label distribution:
count    10726.000000
mean         0.543629
std          0.309978
min          0.010000
25%          0.230000
50%          0.600000
75%          0.830000
max          1.000000
Name: label, dtype: float64

Confidence Score Ranges:
  Very High (0.85-1.0): 2,407 (22.4%)
  High (0.7-0.85): 2,134 (19.9%)
  Medium-High (0.55-0.7): 1,282 (12.0%)
  Medium (0.35-0.55): 1,494 (13.9%)
  Low (0.15-0.35): 1,578 (14.7%)
  Very Low (0.0-0.15): 1,831 (17.1%)

Sample conversations:
  Score 0.92: "What is the temperature outside?"
  Score 0.87: "What’s Tom Brady's career touchdown record?"
  Score 0.83: "What’s the difference between Xbox Series X and S?"
  Score 0.80: "When was DNA discovered?"
  Score 0.67: "Should we binge that new Netflix show everyone is talking about?"
  Score 0.43: "[Partner], did you see the note from [neighbor]?"
  Score 0.24: "I'm so stre

In [4]:
class IntentDataset(Dataset):
    """Dataset class for intent classification."""

    def __init__(self, texts: List[str], labels: List[float], tokenizer, max_length: int = 512):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = float(self.labels[idx])

        encoding = self.tokenizer(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.float)
        }


class ModernBERTIntentClassifier(nn.Module):
    """ModernBERT-based intent classifier for device response decisions."""

    def __init__(self, model_name: str = "answerdotai/ModernBERT-base", dropout_rate: float = 0.3):
        super(ModernBERTIntentClassifier, self).__init__()

        self.config = AutoConfig.from_pretrained(model_name)
        self.bert = AutoModel.from_pretrained(model_name)

        self.dropout = nn.Dropout(dropout_rate)
        self.classifier = nn.Linear(self.config.hidden_size, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask
        )

        # Use [CLS] token representation
        pooled_output = outputs.last_hidden_state[:, 0]
        pooled_output = self.dropout(pooled_output)

        logits = self.classifier(pooled_output)
        probabilities = self.sigmoid(logits)

        loss = None
        if labels is not None:
            loss_fn = nn.MSELoss()
            loss = loss_fn(probabilities.squeeze(), labels)

        # Return format expected by Trainer: (loss, logits) tuple
        if loss is not None:
            return (loss, logits)
        else:
            return logits


In [5]:
## 3. Training Setup and Model Training


In [6]:
# Configuration
MODEL_NAME = "answerdotai/ModernBERT-base"
OUTPUT_DIR = "./intent_classifier_model"
BATCH_SIZE = 16
LEARNING_RATE = 2e-5
NUM_EPOCHS = 4

# Initialize tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Prepare data splits
texts = df['text'].tolist()
labels = df['label'].tolist()

print(f"Preparing train/validation/test splits...")
print(f"Total samples: {len(texts)}")

# Create stratification groups based on confidence ranges for better distribution
def get_strata(label_list):
    """Create stratification groups based on confidence ranges"""
    strata = []
    for label in label_list:
        if label >= 0.8:
            strata.append('very_high')
        elif label >= 0.6:
            strata.append('high')
        elif label >= 0.4:
            strata.append('medium')
        else:
            strata.append('low')
    return strata

# Split into train (70%), validation (15%), test (15%)
train_texts, temp_texts, train_labels, temp_labels = train_test_split(
    texts, labels, test_size=0.3, random_state=42,
    stratify=get_strata(labels)
)

val_texts, test_texts, val_labels, test_labels = train_test_split(
    temp_texts, temp_labels, test_size=0.5, random_state=42,
    stratify=get_strata(temp_labels)
)

# Create datasets
train_dataset = IntentDataset(train_texts, train_labels, tokenizer)
val_dataset = IntentDataset(val_texts, val_labels, tokenizer)
test_dataset = IntentDataset(test_texts, test_labels, tokenizer)

print(f"Training: {len(train_dataset)}, Validation: {len(val_dataset)}, Test: {len(test_dataset)}")

# Show distribution across splits
print(f"\nLabel distributions:")
for split_name, split_labels in [("Train", train_labels), ("Val", val_labels), ("Test", test_labels)]:
    mean_conf = np.mean(split_labels)
    print(f"  {split_name}: Mean confidence = {mean_conf:.3f}")

def compute_metrics(eval_pred: EvalPrediction) -> Dict[str, float]:
    """Compute metrics for regression (confidence prediction) and classification"""
    predictions, labels = eval_pred
    probabilities = torch.sigmoid(torch.tensor(predictions)).numpy()

    # Regression metrics (main task)
    mse = mean_squared_error(labels, probabilities)
    mae = np.mean(np.abs(labels - probabilities))

    # Classification metrics (for analysis)
    # Using 0.6 as threshold (device should respond)
    binary_preds = (probabilities > 0.6).astype(int)
    binary_labels = (labels > 0.6).astype(int)

    accuracy = accuracy_score(binary_labels, binary_preds)
    precision, recall, f1, _ = precision_recall_fscore_support(binary_labels, binary_preds, average='binary', zero_division=0)

    return {
        'mse': mse,
        'mae': mae,
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/20.8k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.13M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

Preparing train/validation/test splits...
Total samples: 10726
Training: 7508, Validation: 1609, Test: 1609

Label distributions:
  Train: Mean confidence = 0.544
  Val: Mean confidence = 0.544
  Test: Mean confidence = 0.543


In [7]:
# Disable wandb logging to avoid API key prompt
import os
import warnings
os.environ["WANDB_DISABLED"] = "true"

# Suppress PyTorch inductor warnings for cleaner output
warnings.filterwarnings("ignore", message=".*Not enough SMs to use max_autotune_gemm mode.*")

# Initialize model and train
model = ModernBERTIntentClassifier(MODEL_NAME)
model.to(device)

training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    num_train_epochs=NUM_EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    learning_rate=LEARNING_RATE,
    warmup_steps=100,
    weight_decay=0.01,
    logging_steps=50,
    eval_strategy="steps", # Changed from evaluation_strategy to eval_strategy
    eval_steps=100,
    save_strategy="steps",
    save_steps=200,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    report_to=[],  # Empty list to disable all reporting
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

print("Starting training...")
trainer.train()
print("Training completed!")

# Evaluate and save
test_results = trainer.evaluate(test_dataset)
print("\nTest Results:")
for metric, value in test_results.items():
    if isinstance(value, float):
        print(f"  {metric}: {value:.4f}")

trainer.save_model(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)
print(f"\nModel saved to {OUTPUT_DIR}")

config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/599M [00:00<?, ?B/s]

Starting training...


/usr/local/lib/python3.11/dist-packages/torch/_inductor/compile_fx.py:194: UserWarning: TensorFloat32 tensor cores for float32 matrix multiplication available but not enabled. Consider setting `torch.set_float32_matmul_precision('high')` for better performance.
  warnings.warn(


Step,Training Loss,Validation Loss,Mse,Mae,Accuracy,Precision,Recall,F1
100,0.048400,0.034635,0.034635,0.343803,0.865134,0.811219,0.947103,0.873911
200,0.036600,0.024642,0.024642,0.336984,0.885643,0.851382,0.930730,0.889290
300,0.026100,0.023673,0.023673,0.363009,0.901802,0.863844,0.950882,0.905276
400,0.021000,0.017776,0.017776,0.352102,0.917340,0.901580,0.934509,0.917749
500,0.018700,0.016414,0.016414,0.346617,0.906774,0.925926,0.881612,0.903226
600,0.016500,0.019374,0.019374,0.348585,0.913611,0.876870,0.959698,0.916416
700,0.016300,0.014278,0.014278,0.348604,0.919826,0.909988,0.929471,0.919626
800,0.017900,0.015557,0.015557,0.338727,0.911746,0.931217,0.886650,0.908387
900,0.014800,0.015533,0.015533,0.348034,0.920447,0.909091,0.931990,0.920398
1000,0.012100,0.015613,0.015613,0.342106,0.920447,0.891765,0.954660,0.922141


Training completed!



Test Results:
  eval_loss: 0.0120
  eval_mse: 0.0120
  eval_mae: 0.3505
  eval_accuracy: 0.9341
  eval_precision: 0.9123
  eval_recall: 0.9583
  eval_f1: 0.9347
  eval_runtime: 14.6482
  eval_samples_per_second: 109.8430
  eval_steps_per_second: 6.8950
  epoch: 4.0000

Model saved to ./intent_classifier_model


In [8]:
## 4. Test the Trained Model


In [15]:
# Test the model with example utterances
def predict_confidence(text, model, tokenizer, device):
    """Predict confidence score for a given text"""
    model.eval()

    encoding = tokenizer(
        text,
        add_special_tokens=True,
        max_length=512,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt'
    )

    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        # Apply sigmoid to get confidence score between 0 and 1
        confidence = torch.sigmoid(outputs).cpu().numpy()[0][0] # Access the single probability value


    return confidence

# Test examples covering different scenarios
test_examples = [
    # Very High Confidence (≥0.85) - Direct commands
    "What's the weather tomorrow?",
    "Set timer for 10 minutes",
    "Turn off the lights",
    "What time does Target close?",

    # High Confidence (≥0.7) - Conversational but helpful
    "[Sarah], where's the nearest bookstore?",
    "Hey [mom], what's the weather like?",
    "[Friend], do you know if that restaurant takes reservations?",
    "I wonder what time the store closes",

    # Medium-High (≥0.55) - Potentially helpful
    "What should we watch tonight?",
    "[Partner], what do you think about ordering pizza?",
    "Should we invite more people to the party?",

    # Medium (0.35-0.55) - Ambiguous
    "How are you feeling about work?",
    "[Name], you look tired today",

    # Low (0.15-0.35) - Personal/emotional
    "I'm so stressed about my boss",
    "[Partner], I love you so much",

    # Very Low (<0.15) - Social chat
    "Thanks for coming over [name]",
    "You look great tonight",
    "What's the weather in this video game world?",
]

print("🧪 Testing trained model with example utterances:")
print("=" * 80)

for text in test_examples:
    confidence = predict_confidence(text, model, tokenizer, device)

    # Determine action based on confidence
    if confidence >= 0.85:
        action = "🔥 VOICE + VISUAL"
    elif confidence >= 0.7:
        action = "🔺 DEFINITELY HELP"
    elif confidence >= 0.55:
        action = "📊 POTENTIALLY HELP"
    elif confidence >= 0.35:
        action = "📉 AMBIGUOUS"
    elif confidence >= 0.15:
        action = "🔻 PROBABLY IGNORE"
    else:
        action = "⬇️ DEFINITELY IGNORE"

    print(f"{confidence:.3f} | {action} | \"{text}\"")

print("=" * 80)
print("✅ Model testing complete! Ready for deployment.")

🧪 Testing trained model with example utterances:
0.896 | 🔥 VOICE + VISUAL | "What's the weather tomorrow?"
0.965 | 🔥 VOICE + VISUAL | "Set timer for 10 minutes"
0.965 | 🔥 VOICE + VISUAL | "Turn off the lights"
0.814 | 🔺 DEFINITELY HELP | "What time does Target close?"
0.449 | 📉 AMBIGUOUS | "[Sarah], where's the nearest bookstore?"
0.403 | 📉 AMBIGUOUS | "Hey [mom], what's the weather like?"
0.377 | 📉 AMBIGUOUS | "[Friend], do you know if that restaurant takes reservations?"
0.579 | 📊 POTENTIALLY HELP | "I wonder what time the store closes"
0.680 | 📊 POTENTIALLY HELP | "What should we watch tonight?"
0.346 | 🔻 PROBABLY IGNORE | "[Partner], what do you think about ordering pizza?"
0.462 | 📉 AMBIGUOUS | "Should we invite more people to the party?"
0.286 | 🔻 PROBABLY IGNORE | "How are you feeling about work?"
0.200 | 🔻 PROBABLY IGNORE | "[Name], you look tired today"
0.116 | ⬇️ DEFINITELY IGNORE | "I'm so stressed about my boss"
0.140 | ⬇️ DEFINITELY IGNORE | "[Partner], I love you so much"

In [10]:
## 5. Download Trained Model

# Download ONLY the final model files (not checkpoints)
import zipfile
from google.colab import files

zip_filename = 'intent_classifier_modernbert.zip'
print(f"📦 Packaging FINAL model only (excluding checkpoints)...")

# List of essential files to include (checking both formats)
essential_files = [
    'pytorch_model.bin',           # Main model weights (PyTorch format)
    'model.safetensors',           # Main model weights (SafeTensors format)
    'config.json',                 # Model configuration
    'tokenizer_config.json',       # Tokenizer config
    'tokenizer.json',              # Tokenizer
    'vocab.txt',                   # Vocabulary (if exists)
    'special_tokens_map.json',     # Special tokens
    'training_args.bin'            # Training arguments
]

# First, let's see what files are actually in the directory
print(f"🔍 Files in {OUTPUT_DIR}:")
for file in os.listdir(OUTPUT_DIR):
    if os.path.isfile(os.path.join(OUTPUT_DIR, file)):
        file_size = os.path.getsize(os.path.join(OUTPUT_DIR, file)) / (1024*1024)
        print(f"  📄 {file} ({file_size:.1f} MB)")

print(f"\n📦 Packaging files...")

with zipfile.ZipFile(zip_filename, 'w') as zipf:
    files_added = 0
    total_size = 0

    # Add all non-checkpoint files
    for file in os.listdir(OUTPUT_DIR):
        file_path = os.path.join(OUTPUT_DIR, file)
        if os.path.isfile(file_path) and not file.startswith('checkpoint'):
            file_size = os.path.getsize(file_path)
            zipf.write(file_path, f"intent_classifier_model/{file}")
            print(f"  ✅ Added: {file} ({file_size/(1024*1024):.1f} MB)")
            files_added += 1
            total_size += file_size

    print(f"\n📊 Summary: {files_added} files added, total {total_size/(1024*1024):.1f} MB")

# Check file size
zip_size = os.path.getsize(zip_filename)
zip_size_mb = zip_size / (1024 * 1024)

print(f"✅ Created {zip_filename}")
print(f"📦 File size: {zip_size_mb:.1f} MB ({zip_size:,} bytes)")
print(f"📁 Contains: Model weights, tokenizer, config files")
print(f"🚀 Ready for deployment in your always-listening AI device!")
print(f"💾 Downloading...")

files.download(zip_filename)

print(f"\n🎉 Training and download complete!")
print(f"\nNext steps:")
print(f"1. Extract the model files from the ZIP")
print(f"2. Use the model in your AI device pipeline")
print(f"3. Adjust confidence thresholds based on testing")
print(f"4. Test with real conversations and iterate")

📦 Packaging FINAL model only (excluding checkpoints)...
🔍 Files in ./intent_classifier_model:
  📄 special_tokens_map.json (0.0 MB)
  📄 model.safetensors (568.5 MB)
  📄 tokenizer_config.json (0.0 MB)
  📄 tokenizer.json (3.4 MB)
  📄 training_args.bin (0.0 MB)

📦 Packaging files...
  ✅ Added: special_tokens_map.json (0.0 MB)
  ✅ Added: model.safetensors (568.5 MB)
  ✅ Added: tokenizer_config.json (0.0 MB)
  ✅ Added: tokenizer.json (3.4 MB)
  ✅ Added: training_args.bin (0.0 MB)

📊 Summary: 5 files added, total 571.9 MB
✅ Created intent_classifier_modernbert.zip
📦 File size: 571.9 MB (599,685,209 bytes)
📁 Contains: Model weights, tokenizer, config files
🚀 Ready for deployment in your always-listening AI device!
💾 Downloading...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


🎉 Training and download complete!

Next steps:
1. Extract the model files from the ZIP
2. Use the model in your AI device pipeline
3. Adjust confidence thresholds based on testing
4. Test with real conversations and iterate


In [16]:
import os

OUTPUT_DIR = "./intent_classifier_model"

print(f"🔍 Full list of files in {OUTPUT_DIR}:")
if os.path.exists(OUTPUT_DIR) and os.path.isdir(OUTPUT_DIR):
    for file in os.listdir(OUTPUT_DIR):
        file_path = os.path.join(OUTPUT_DIR, file)
        if os.path.isfile(file_path):
            file_size = os.path.getsize(file_path) / (1024*1024) # size in MB
            print(f"  📄 {file} ({file_size:.1f} MB)")
else:
    print(f"Error: Directory '{OUTPUT_DIR}' not found.")

🔍 Full list of files in ./intent_classifier_model:
  📄 special_tokens_map.json (0.0 MB)
  📄 model.safetensors (568.5 MB)
  📄 tokenizer_config.json (0.0 MB)
  📄 tokenizer.json (3.4 MB)
  📄 training_args.bin (0.0 MB)


In [ ]:
# Interactive testing function
def interactive_predict(model, tokenizer, device):
    """Allows interactive testing of the model with user input."""
    print("Enter text to test the model (type 'quit' to exit):")

    while True:
        try:
            user_input = input("> ")
            if user_input.lower() == 'quit':
                break

            if not user_input:
                print("Please enter some text.")
                continue

            confidence = predict_confidence(user_input, model, tokenizer, device)

            # Determine action based on confidence (using the same thresholds as before)
            if confidence >= 0.85:
                action = "🔥 VOICE + VISUAL"
            elif confidence >= 0.7:
                action = "🔺 DEFINITELY HELP"
            elif confidence >= 0.55:
                action = "📊 POTENTIALLY HELP"
            elif confidence >= 0.35:
                action = "📉 AMBIGUOUS"
            elif confidence >= 0.15:
                action = "🔻 PROBABLY IGNORE"
            else:
                action = "⬇️ DEFINITELY IGNORE"

            print(f"  Confidence: {confidence:.3f} | Action: {action}")

        except Exception as e:
            print(f"An error occurred: {e}")
            # This might happen if there's an issue with tokenization or model inference
            print("Please try entering the text again.")

# To run interactive testing:
# Ensure the model, tokenizer, and device are loaded (from previous cells)
interactive_predict(model, tokenizer, device)

print("To start interactive testing, uncomment and run the last line:")
print("# interactive_predict(model, tokenizer, device)")

Enter text to test the model (type 'quit' to exit):
> where should I eat tomorrow
  Confidence: 0.580 | Action: 📊 POTENTIALLY HELP
> whats the best restuarant around here
  Confidence: 0.737 | Action: 🔺 DEFINITELY HELP
